In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shanegerami/ai-vs-human-text")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'ai-vs-human-text' dataset.
Path to dataset files: /kaggle/input/ai-vs-human-text


In [2]:
import pandas as pd
import os

file_path = os.path.join(path, 'AI_Human.csv')

df = pd.read_csv(file_path)

df.head()

# 0 -> Human
# 1 -> AI

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [3]:
df = df.rename(columns={'generated': 'labels'})

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
label_col = 'labels'
text_col = 'text'

In [6]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^a-zA-Z\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    tokens = [word for word in text.split() if word not in stop_words]
    return " ".join(tokens)

df['clean_text'] = df[text_col].apply(clean_text)

In [7]:
df.head()

,text,labels,clean_text
0,Cars. Cars have been around since they became ...,0.0,cars cars around since became famous henry for...
1,Transportation is a large necessity in most co...,0.0,transportation large necessity countries world...
2,"""America's love affair with it's vehicles seem...",0.0,americas love affair vehicles seems cooling sa...
3,How often do you ride in a car? Do you drive a...,0.0,often ride car drive one motor vehicle work st...
4,Cars are a wonderful thing. They are perhaps o...,0.0,cars wonderful thing perhaps one worlds greate...


In [8]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import os

df = df.drop("text", axis='columns')

In [9]:
if df[label_col].dtype == object:
    unique_labels = df[label_col].unique().tolist()
    print("unique labels:", unique_labels)
    label2id = {lab: i for i, lab in enumerate(sorted(unique_labels))}
    print("label2id:", label2id)
    df[label_col] = df[label_col].map(label2id)

In [10]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Distillation

In [11]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [12]:
cd drive/MyDrive/Colab Notebooks/WeightsNLP

/content/drive/MyDrive/Colab Notebooks/WeightsNLP


In [13]:
!pip install transformers datasets accelerate -q
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    get_scheduler
)
from torch.optim import AdamW
from datasets import load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
output_dir = "bert_finetuned"

In [15]:
student_model_name = "distilbert-base-uncased"

teacher = AutoModelForSequenceClassification.from_pretrained(output_dir).to(device)
teacher.eval()
for param in teacher.parameters():
    param.requires_grad = False

student = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=2).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
dataset = dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["clean_text"], truncation=True, padding="max_length", max_length=256)

encoded_dataset = dataset.map(tokenize, batched=True)
if "label" in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/389788 [00:00<?, ? examples/s]

Map:   0%|          | 0/97447 [00:00<?, ? examples/s]

In [17]:
small_train = encoded_dataset["train"].shuffle(seed=42).select(range(10000))
small_test  = encoded_dataset["test"].shuffle(seed=42).select(range(4000))

train_loader = DataLoader(small_train, batch_size=32, shuffle=True)
test_loader  = DataLoader(small_test, batch_size=32)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DistillationLoss(nn.Module):
    """
    A stable and flexible Knowledge Distillation loss implementation.
    Supports:
      - Hard label loss (cross-entropy)
      - Soft label (KL-divergence)
      - Feature-level distillation
      - Relation-based distillation
    """
    def __init__(self, temperature=2.0, alpha=0.5, beta=0.0, gamma=0.0, layer_mapping=None):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha       # Hard label (supervised) weight
        self.beta = beta         # Feature distillation weight
        self.gamma = gamma       # Relation distillation weight
        self.layer_mapping = layer_mapping
        self.ce = nn.CrossEntropyLoss()

    def forward(self, student_outputs, teacher_outputs, labels):
        """
        Args:
            student_outputs: ModelOutput from student (logits + hidden_states)
            teacher_outputs: ModelOutput from teacher (logits + hidden_states)
            labels: Ground truth labels (LongTensor)
        """
        student_logits = student_outputs.logits
        teacher_logits = teacher_outputs.logits

        # ---------- 1. Hard-label loss ----------
        hard_loss = self.ce(student_logits, labels)

        # ---------- 2. Soft-label (KL) distillation loss ----------
        T = self.temperature
        with torch.no_grad():
            teacher_soft = F.softmax(teacher_logits / T, dim=-1)
        student_log_soft = F.log_softmax(student_logits / T, dim=-1)
        soft_loss = F.kl_div(student_log_soft, teacher_soft, reduction="batchmean") * (T * T)

        # ---------- 3. Feature distillation ----------
        feature_loss = 0.0
        if self.beta > 0 and self.layer_mapping and hasattr(student_outputs, "hidden_states"):
            student_hs = student_outputs.hidden_states
            teacher_hs = teacher_outputs.hidden_states
            for s_layer, t_layer in self.layer_mapping.items():
                s_feat = student_hs[s_layer]
                t_feat = teacher_hs[t_layer]
                feature_loss += F.mse_loss(s_feat, t_feat.detach())
            feature_loss = feature_loss / len(self.layer_mapping)  # normalize

        # ---------- 4. Relation distillation ----------
        relation_loss = 0.0
        if self.gamma > 0 and hasattr(student_outputs, "hidden_states"):
            s_last = student_outputs.hidden_states[-1]
            t_last = teacher_outputs.hidden_states[-1]
            # Normalize to avoid explosion in dot product
            s_norm = F.normalize(s_last, p=2, dim=-1)
            t_norm = F.normalize(t_last, p=2, dim=-1)
            s_relation = torch.bmm(s_norm, s_norm.transpose(1, 2))
            t_relation = torch.bmm(t_norm, t_norm.transpose(1, 2))
            relation_loss = F.mse_loss(s_relation, t_relation.detach())

        # ---------- 5. Total loss ----------
        # Keep total loss scale balanced, so we don't require α+β+γ=1 necessarily.
        total_loss = (
            self.alpha * hard_loss
            + (1 - self.alpha - self.beta - self.gamma) * soft_loss
            + self.beta * feature_loss
            + self.gamma * relation_loss
        )

        return total_loss, {
            "hard_loss": hard_loss.item(),
            "soft_loss": soft_loss.item(),
            "feature_loss": feature_loss if isinstance(feature_loss, float) else feature_loss.item(),
            "relation_loss": relation_loss if isinstance(relation_loss, float) else relation_loss.item(),
        }


In [19]:
distillation_config = {
    "temperature": 3.0,
    "alpha": 0.7,                 # soft label weight
    "beta": 0.2,                  # feature weight
    "gamma": 0.1,                 # Slight relation weight

    # DistilBERT layer → BERT layer
    "layer_mapping": {
        0: 0,
        1: 2,
        2: 4,
        3: 6,
        4: 8,
        5: 10
    },

    "batch_size": 8,
    "gradient_accumulation_steps": 4,
    "learning_rate": 3e-5,
    "weight_decay": 0.01,
    "epochs": 10,
    "warmup_ratio": 0.1,
}

optimizer = AdamW(student.parameters(), lr=distillation_config["learning_rate"], weight_decay=distillation_config["weight_decay"])
num_training_steps = len(train_loader) // distillation_config["gradient_accumulation_steps"] * distillation_config["epochs"]
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=int(num_training_steps * distillation_config["warmup_ratio"]), num_training_steps=num_training_steps)

# Ensure student and teacher models return hidden states
student.config.output_hidden_states = True
teacher.config.output_hidden_states = True


total_epochs = distillation_config["epochs"]

for epoch in range(total_epochs):
    # Progressive Distillation: Adjust loss weights based on epoch
    if epoch < total_epochs * 0.33:  # Stage 1: Basic concept learning
        current_alpha = 0.2  # Higher soft label weight
        current_beta = 0.0   # No feature distillation
        current_gamma = 0.0  # No relation distillation
        current_temperature = 5.0 # Higher temperature
    elif epoch < total_epochs * 0.66:  # Stage 2: Feature learning
        current_alpha = 0.2  # Reduced soft label weight
        current_beta = 0.2  # Feature distillation
        current_gamma = 0.0  # Slight relation distillation
        current_temperature = 3.0 # Medium temperature
    else:  # Stage 3: Comprehensive knowledge fusion
        current_alpha = 0.2  # Balanced weights
        current_beta = 0.2   # Stronger feature distillation
        current_gamma = 0.2  # Stronger relation distillation
        current_temperature = 2.0 # Lower temperature

    loss_fn = DistillationLoss(
        temperature=current_temperature,
        alpha=current_alpha,
        beta=current_beta,
        gamma=current_gamma,
        layer_mapping=distillation_config["layer_mapping"] if current_beta > 0 else None # Only use layer mapping if beta > 0
    )


    student.train()
    total_loss = 0
    # Reset gradients every accumulation steps
    optimizer.zero_grad()
    for step, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            teacher_outputs = teacher(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], output_hidden_states=True)

        student_outputs = student(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], output_hidden_states=True)

        loss, _ = loss_fn(student_outputs, teacher_outputs, batch["labels"].long()) # Cast labels to long and unpack the tuple
        loss = loss / distillation_config["gradient_accumulation_steps"] # Normalize loss for accumulation

        loss.backward()

        if (step + 1) % distillation_config["gradient_accumulation_steps"] == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad() # Clear gradients after optimization step

        total_loss += loss.item() * distillation_config["gradient_accumulation_steps"] # Scale back loss for reporting


    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 2.1123
Epoch 2, Loss: 0.2306
Epoch 3, Loss: 0.1092
Epoch 4, Loss: 0.0753
Epoch 5, Loss: 0.1050
Epoch 6, Loss: 0.0981
Epoch 7, Loss: 0.0921
Epoch 8, Loss: 0.0858
Epoch 9, Loss: 0.0819
Epoch 10, Loss: 0.0810


In [20]:
import time, torch
import psutil
from sklearn.metrics import accuracy_score

def evaluate_with_speed(model, dataloader, name="model", warmup=10):
    model.eval()
    all_preds, all_labels = [], []

    # ====== speed ======
    start_time = time.time()
    total_samples = 0

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}

            if i == warmup:
                start_time = time.time()
                total_samples = 0

            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            preds = torch.argmax(outputs.logits, dim=-1)

            total_samples += batch["input_ids"].size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())

    elapsed = time.time() - start_time
    speed = total_samples / elapsed

    # ====== acc ======
    acc = accuracy_score(all_labels, all_preds)

    # ====== mem ======
    if torch.cuda.is_available():
        mem = torch.cuda.max_memory_allocated(device) / (1024**2)  # MB
        torch.cuda.reset_peak_memory_stats()
    else:
        mem = psutil.Process().memory_info().rss / (1024**2)  # MB (CPU)

    print(f"{name} | Acc: {acc:.4f} | Speed: {speed:.2f} samples/sec | Mem: {mem:.2f} MB")
    return acc, speed, mem

In [21]:
# Teacher
teacher_acc, teacher_speed, teacher_mem = evaluate_with_speed(teacher, test_loader, "Teacher")

# Student (before distill)
raw_student = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)
raw_acc, raw_speed, raw_mem = evaluate_with_speed(raw_student, test_loader, "Student (before distill)")

# Student (after distill)
student_acc, student_speed, student_mem = evaluate_with_speed(student, test_loader, "Student (after distill)")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Teacher | Acc: 0.9695 | Speed: 69.50 samples/sec | Mem: 4467.06 MB
Student (before distill) | Acc: 0.3705 | Speed: 139.31 samples/sec | Mem: 2243.08 MB
Student (after distill) | Acc: 0.9712 | Speed: 139.22 samples/sec | Mem: 2507.08 MB
